In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pandas as pd

In [160]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\ubc\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [19]:
# Define url  and set up config splinter to the site 
#Create a function that takes the url and return the soup 
def create_soup(url):
    browser.visit(url)
    # Create BeautifulSoup object; parse with 'html.parser'
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    return soup

### -------------------------------------------------------------------------------------------------------------------------------
#### NASA Mars News
    - Scrape the Mars News Site and collect the latest News Title and Paragraph Text.
### -------------------------------------------------------------------------------------------------------------------------------

In [23]:
#Define the url to visit and call the function create_soup
url = 'https://redplanetscience.com/'
soup = create_soup(url)

In [24]:
news_title =[]
content_title = soup.find_all('div' , class_="content_title")
for title in content_title:
    news_title.append(title.text)
news_title

["NASA's Mars 2020 Will Hunt for Microscopic Fossils",
 "NASA's Perseverance Rover Bringing 3D-Printed Metal Parts to Mars",
 "Mars Helicopter Attached to NASA's Perseverance Rover",
 'NASA Invites Students to Name Mars 2020 Rover',
 'Naming a NASA Mars Rover Can Change Your Life',
 "Follow NASA's Perseverance Rover in Real Time on Its Way to Mars",
 "NASA's Perseverance Rover Will Peer Beneath Mars' Surface ",
 'NASA Wins Two Emmy Awards for Interactive Mission Coverage',
 'MOXIE Could Help Future Rockets Launch Off Mars',
 'NASA Adds Return Sample Scientists to Mars 2020 Leadership Team',
 "NASA's Mars Reconnaissance Orbiter Undergoes Memory Update",
 'NASA, ULA Launch Mars 2020 Perseverance Rover Mission to Red Planet',
 "NASA's Curiosity Mars Rover Snaps Its Highest-Resolution Panorama Yet",
 "NASA-JPL Names 'Rolling Stones Rock' on Mars",
 "Air Deliveries Bring NASA's Perseverance Mars Rover Closer to Launch"]

In [25]:
news_p =[]
content_para = soup.find_all('div', class_="article_teaser_body")
for para in content_para:
    news_p.append(para.text)
news_p

["A new paper identifies a ring of minerals at the rover's landing site that are ideal for fossilizing microbial life.",
 "For hobbyists and makers, 3D printing expands creative possibilities; for specialized engineers, it's also key to next-generation spacecraft design.",
 "The team also fueled the rover's sky crane to get ready for this summer's history-making launch.",
 "Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover.",
 'Want to name the robotic scientist NASA is sending to Mars in 2020? The student who named Curiosity — the rover currently exploring Mars — will tell you this is an opportunity worth taking.',
 "A crisply rendered web application can show you where the agency's Mars 2020 mission is right now as it makes its way to the Red Planet for a Feb. 18, 2021, landing.",
 "The agency's newest rover will use the first ground-penetrating radar instrument on the Martian surface to help search for signs of past mic

### -------------------------------------------------------------------------------------------------------------------------------
#### JPL Mars Space Images - Featured Image
    - Visit the url for the Featured Space Image site https://spaceimages-mars.com/
    - Find the image url to the full size .jpg image.
### -------------------------------------------------------------------------------------------------------------------------------

In [26]:
# Define url  and call create_soup function to return soup 
featured_url = 'https://spaceimages-mars.com/'
soup = create_soup(featured_url)

In [32]:
# Create BeautifulSoup object; parse with 'html.parser' to the second site
try:
    target = 'button[class="btn btn-outline-light"]'
    browser.find_by_tag(target).click()
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    image_src = soup.find('img', class_="fancybox-image")['src']
except:
    print('can\'t find the image')
featured_image_url = featured_url + image_src
featured_image_url

'https://spaceimages-mars.com/image/featured/mars3.jpg'

### -------------------------------------------------------------------------------------------------------------------------------
#### Mars Facts
        - Visit the Mars Facts webpage https://galaxyfacts-mars.com/ and use Pandas to scrape the table 
          containing facts about the planet including Diameter, Mass, etc.
        - Use Pandas to convert the data to a HTML table string.
### -------------------------------------------------------------------------------------------------------------------------------

In [40]:
# define url to visit
mars_facts_url = 'https://galaxyfacts-mars.com/'
# using pandas read html
tables = pd.read_html(mars_facts_url)
# find total no. of tables in the list
len(tables)

2

In [46]:
# check for the table in iterest and select that table
mars_fact_df = tables[0]
mars_fact_df

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [49]:
# add column names ['Description' , 'Mars', 'Earth']
mars_fact_df.columns = ['Description' , 'Mars', 'Earth']

# set Description as a index
mars_fact_df.set_index('Description' , drop=True , inplace= True)
mars_fact_df

,Mars,Earth
Description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [ ]:
# Save the table in HTML format
mars_fact_df.to_html('table.html')


### -------------------------------------------------------------------------------------------------------------------------------
#### Mars Hemispheres
          - Visit the astrogeology site here to obtain high resolution images for each of Mar's hemispheres.
          - get image url to the full resolution image.
### -------------------------------------------------------------------------------------------------------------------------------

In [166]:
# Define the url and call create_soup function to return soup object
mars_hem_url = 'https://marshemispheres.com/'
soup = create_soup(mars_hem_url)

container = soup.find_all('div' , class_="item")
for content in container:
    link = content.find('a')['href']
    target = f'a[class="{link}"]'
    print(link)
    browser.links.find_by_partial_href(str(link)).click()
    

    


cerberus.html


ElementNotInteractableException: Message: element not interactable: element has zero size
  (Session info: chrome=89.0.4389.90)


In [ ]:

        browser.links.find_by_href(target).click()        
        # get html of rendered page adn create a soup
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        # Get the title and the image link
        title = soup.find('h2' , class_="title").text
        
        
         try:
        # To get the full resolution image, click the thumbnail image
        target = 'img[class="thumb"]'
        browser.find_by_tag(target).click()          
        # get html of rendered page adn create a soup
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        # Get the title and the image link
        title_hem = soup.find('h2' , class_="title").text
        print(title_hem)
        
    except:
        print('Image not found')
        
    